In [1]:
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
import numpy as np

In [2]:
LFP = Structure.from_file("/data/kevinxhan/dist_chgnet/LiFePO4.cif")
LFP.make_supercell((8, 2, 2))
LFP_atoms = AseAtomsAdaptor().get_atoms(LFP)
print(len(LFP_atoms))

896


In [3]:
import matgl
from DistMLIP.implementations.matgl import Potential_Dist, PESCalculator_Dist
from DistMLIP.implementations.matgl import CHGNet_Dist, TensorNet_Dist
from DistMLIP.implementations.matgl import MolecularDynamics, Relaxer


model = matgl.load_model('/data/kevinxhan/dist_chgnet/matgl-private/pretrained_models/TensorNet-MatPES-PBE-v2025.1-PES').model
dist_model = TensorNet_Dist.from_existing(model)
dist_model.enable_distributed_mode([5,7])
model.eval()


/data/kevinxhan/miniconda3/envs/matgl3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TensorNet(
  (bond_expansion): BondExpansion(
    (rbf): SphericalBesselFunction()
  )
  (activation): SiLU()
  (tensor_embedding): TensorEmbedding(
    (distance_proj1): Linear(in_features=3, out_features=128, bias=True)
    (distance_proj2): Linear(in_features=3, out_features=128, bias=True)
    (distance_proj3): Linear(in_features=3, out_features=128, bias=True)
    (emb): Embedding(89, 128)
    (act): SiLU()
    (linears_tensor): ModuleList(
      (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
    )
    (linears_scalar): ModuleList(
      (0): Linear(in_features=128, out_features=256, bias=True)
      (1): Linear(in_features=256, out_features=384, bias=True)
    )
    (init_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (emb2): Linear(in_features=256, out_features=128, bias=True)
    (emb3): Linear(in_features=3, out_features=128, bias=True)
  )
  (layers): ModuleList(
    (0-1): 2 x TensorNetInteraction(
      (linears_scalar): ModuleList(
     

In [4]:
pes = Potential_Dist(model=dist_model, num_threads=256)
pes(LFP_atoms)

(tensor(-1470.2537, device='cuda:5', grad_fn=<AddBackward0>),
 tensor([[-4.2094e-07,  1.8048e-07,  4.0704e-07],
         [-2.6802e-07,  3.8306e-07,  1.2917e-07],
         [-3.2687e-07,  1.1205e-08,  3.6330e-07],
         ...,
         [-6.5795e-02,  1.8291e-01,  1.1454e-01],
         [-6.5794e-02,  1.8291e-01,  1.1454e-01],
         [-6.5794e-02,  1.8291e-01,  1.1454e-01]], device='cuda:5'),
 tensor([[-1.2942e+00, -3.0518e-07, -1.1524e-06],
         [ 4.3059e-08, -1.0939e+00, -1.8789e-07],
         [-5.1540e-07, -2.3421e-07, -7.9690e-01]], device='cuda:5'),
 None)

In [ ]:
pes = Potential_Dist(model=dist_model, num_threads=256)

driver = MolecularDynamics(
    LFP_atoms,
    potential=pes,
    timestep=0.5,
    temperature=300,
    # logfile="H2O_0.1M_1000k.log",
    # trajectory="H2O_0.1M_1000k.traj",
    loginterval=200
)

driver.run(20)